In [3]:
#import sys
#!{sys.executable} -m pip install yael

### Description based on Fischer Vectors
* We have the image files and corresponding SIFT files
* Global Fisher vector is calculated for each image using SIFTs of the images
* Later, L2 distance between FVs is good to approximate the similarity of the contents of the images


### Algorithm

* list of available images
* load the SIFTs for these images
* Next we sample the descriptors to reduce their dimensionality by PCA and computing a GMM. For a GMM of size k (let’s set it to 64), we need about 1000*k training descriptors

    * make a big matrix with all image descriptors
    * choose n_sample descriptors at random
    * until now sample was in uint8. Convert to float32
    * compute mean and covariance matrix for the PCA
    * compute PCA matrix and keep only 64 dimensions
        sort by increasing eigenvalue
        eigenvectors for the 64 last eigenvalues
    * transform sample with PCA (note that numpy imposes line-vectors,# so we right-multiply the vectors)
    * train GMM (The FV computation relies on a training where a Gaussian Mixture Model (GMM) is fitted to a set of 
        representative local descriptors. For simplicity, we are going to use the descriptors of the database we   index.)
        
* The gmm is a tuple containing the a-priori weights per mixture component, the mixture centres and the diagonal of the component covariance matrices (the model assumes a diagonal matrix, otherwise the descriptor would be way too long).

*  The next stage is to encode the SIFTs into one vector per image. We choose to include only the derivatives     w.r.t. mu in the FV, which results in a FV of size k * 64
    * apply the PCA to the image descriptor
    * compute the Fisher vector, using only the derivative w.r.t mu
    * make one matrix with all FVs
    * normalizations are done on all descriptors at once
    * power-normalization
    * L2 normalize
    * handle images with 0 local descriptor (100 = far away from "normal" images)
    
* Now the FV can be used to compare images, so we compute for each query image the nearest images in the matrix(image_fvs).
    * get the indices of the query images (the subset of images that end in "00")
    * corresponding descriptors
    * get the 8 nearest neighbours for all query images in the image_fvs array
   
* mAP performance for the search
    * collect the positive results in the dataset
    * the positives have the same prefix as the query image
    * ranks of positives. We skip the result #0, assumed to be the query image
    * accumulate trapezoids with this basis

## Brief outline of fisher vectors

#### Vocabulary learning with GMM:
* Sample many features from input images.
* Fit a Gaussian Mixture Model on those features.
* The result is a vocabulary of dominant features in the image, and their distributions.

#### Image representation based on the vocabulary:
* Measure the expectation of the difference and distance of the image features, from each Gaussian distrubution, using the likelihood a feature belongs to certain gaussian.
* Concatenate the resulting vector for each vocabulary word, into one large descriptor vector.